In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
import re
import pandas as pd
import numpy as np
from tqdm import tqdm


In [32]:
def login_and_scrape(input_url, login_url, username, password):
    # Setup Selenium WebDriver with Chrome
    service = Service(executable_path=ChromeDriverManager().install())
    chrome_options = Options()
    ua = UserAgent()
    user_agent = ua.random
    chrome_options.add_argument(f'user-agent={user_agent}')
    # chrome_options.add_argument("--headless")  # Remove this line to see the browser
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        # Navigate to the login page
        driver.get(login_url)

        
        # Wait for successful login (this can be checking for a specific element visible after login)
        WebDriverWait(driver, 100)
        
        # Navigate to the target page for scraping
        driver.get(input_url)
        
        staleness_counter = 0
        links = []
        while staleness_counter <= 100:
            
            # extract details inside class="datatable-row-wrapper" for all rows in class="datatable-scroll" table
            try:                                
                # Attempt to find and click "Load More" button
                rows = driver.find_elements(By.CLASS_NAME, "datatable-row-wrapper")
                
                for row in rows:
                    # Check if the row contains the delete (trash) icon
                    trash_icon = row.find_elements(By.XPATH, ".//i[contains(@class, 'fa-trash-alt')]")
                    if trash_icon:
                        cells = row.find_elements(By.CLASS_NAME, "datatable-body-cell")
                        dealer_info = {
                            "Name": cells[0].text.strip(),
                            "Phone": cells[1].text.strip(),
                            "Status": cells[2].text.strip(),
                            "Type": cells[3].text.strip(),
                            "Street": cells[4].text.strip(),
                            "City": cells[5].text.strip(),
                            "State": cells[6].text.strip(),
                            "Zip": cells[7].text.strip(),
                        }
                        links.append(dealer_info)
                    
                df = pd.DataFrame(links)
                df.to_csv("dealer.csv")
                                

            except (NoSuchElementException, TimeoutException):
                break
            except StaleElementReferenceException:
                staleness_counter += 1
                continue
        

        
    
    finally:
        # Ensure the driver is closed properly
        driver.quit()



In [33]:
# Example usage
login_url = "http://m2-angular-ui.s3-website.us-east-2.amazonaws.com/authentication/login"
input_url = "http://m2-angular-ui.s3-website.us-east-2.amazonaws.com/CRM/Dealers"
username = "officialfavasm@gmail.com"
password = "8r5mDAvPMLjetSX"

df = login_and_scrape(input_url, login_url, username, password)


KeyboardInterrupt: 